# Hitchhiker's Guide to Hyperspace (Scala)
## An Indexing Subsystem for Apache Spark™

<img src="https://raw.githubusercontent.com/rapoth/hyperspace/master/docs/assets/images/hyperspace-small-banner.png" alt="Hyperspace Indexing Sub-System Logo" width="1000"/>

[Hyperspace](https://github.com/microsoft/hyperspace) introduces the ability for Apache Spark™ users to create indexes on their datasets (e.g., CSV, JSON, Parquet etc.) and leverage them for potential query and workload acceleration.

In this notebook, we highlight the basics of Hyperspace, emphasizing on its simplicity and show how it can be used by just anyone.

**Disclaimer**: Hyperspace helps accelerate your workloads/queries under two circumstances:

  1. Queries contain filters on predicates with high selectivity (e.g., you want to select 100 matching rows from a million candidate rows)
  2. Queries contain a join that requires heavy-shuffles (e.g., you want to join a 100 GB dataset with a 10 GB dataset)

You may want to carefully monitor your workloads and determine whether indexing is helping you on a case-by-case basis.

## Setup
To begin with, let's start a new Spark™ session. Since this notebook is a tutorial merely to illustrate what Hyperspace can offer, we will make a configuration change that allow us to highlight what Hyperspace is doing on small datasets. By default, Spark™ uses *broadcast join* to optimize join queries when the data size for one side of join is small (which is the case for the sample data we use in this tutorial). Therefore, we disable broadcast joins so that later when we run join queries, Spark™ uses *sort-merge* join. This is mainly to show how Hyperspace indexes would be used at scale for accelerating join queries.

The output of running the cell below shows a reference to the successfully created Spark™ session and prints out '-1' as the value for the modified join config which indicates that broadcast join is successfully disabled.

In [2]:
val sessionId = scala.util.Random.nextInt(1000000)
val dataPath = s"/hyperspace/data-$sessionId";
val indexLocation = s"/hyperspace/indexes-$sessionId"

// Please note that you DO NOT need to change this configuration in production.
// We store all indexes in the system folder within Synapse.
spark.conf.set("spark.hyperspace.system.path", indexLocation)

StatementMeta(small, 17, 4, Finished, Available)

sessionId: Int = 269515
dataPath: String = /hyperspace/data-269515
indexLocation: String = /hyperspace/indexes-269515


In [3]:
// Start your Spark™ session
spark

// Disable BroadcastHashJoin, so Spark™ will use standard SortMergeJoin. Currently hyperspace indexes utilize SortMergeJoin to speed up query.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

// Verify that BroadcastHashJoin is set correctly 
println(spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))
spark.conf.set("spark.hyperspace.explain.displayMode", "html")

StatementMeta(small, 17, 5, Finished, Available)

res13: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ab7b5f
-1


## Data Preparation

To prepare our environment, we will create sample data records and save them as parquet data files. While we use Parquet for illustration, you can use other formats such as CSV. In the subsequent cells, we will also demonstrate how you can create several Hyperspace indexes on this sample dataset and how one can make Spark™ use them when running queries. 

Our example records correspond to two datasets: *department* and *employee*. You should configure "empLocation" and "deptLocation" paths so that on the storage account they point to your desired location to save generated data files. 

The output of running below cell shows contents of our datasets as lists of triplets followed by references to dataFrames created to save the content of each dataset in our preferred location.

In [4]:
import org.apache.spark.sql.DataFrame

// Sample department records
val departments = Seq(
      (10, "Accounting", "New York"),
      (20, "Research", "Dallas"),
      (30, "Sales", "Chicago"),
      (40, "Operations", "Boston"))

// Sample employee records
val employees = Seq(
      (7369, "SMITH", 20),
      (7499, "ALLEN", 30),
      (7521, "WARD", 30),
      (7566, "JONES", 20),
      (7698, "BLAKE", 30),
      (7782, "CLARK", 10),
      (7788, "SCOTT", 20),
      (7839, "KING", 10),
      (7844, "TURNER", 30),
      (7876, "ADAMS", 20),
      (7900, "JAMES", 30),
      (7934, "MILLER", 10),
      (7902, "FORD", 20),
      (7654, "MARTIN", 30))

// Save sample data in the Parquet format
import spark.implicits._
val empData: DataFrame = employees.toDF("empId", "empName", "deptId")
val deptData: DataFrame = departments.toDF("deptId", "deptName", "location")

val empLocation: String = s"$dataPath/employees.parquet"
val deptLocation: String = s"$dataPath/departments.parquet"
empData.write.mode("overwrite").parquet(empLocation)
deptData.write.mode("overwrite").parquet(deptLocation)

StatementMeta(small, 17, 6, Finished, Available)

import org.apache.spark.sql.DataFrame
departments: Seq[(Int, String, String)] = List((10,Accounting,New York), (20,Research,Dallas), (30,Sales,Chicago), (40,Operations,Boston))
employees: Seq[(Int, String, Int)] = List((7369,SMITH,20), (7499,ALLEN,30), (7521,WARD,30), (7566,JONES,20), (7698,BLAKE,30), (7782,CLARK,10), (7788,SCOTT,20), (7839,KING,10), (7844,TURNER,30), (7876,ADAMS,20), (7900,JAMES,30), (7934,MILLER,10), (7902,FORD,20), (7654,MARTIN,30))
import spark.implicits._
empData: org.apache.spark.sql.DataFrame = [empId: int, empName: string ... 1 more field]
deptData: org.apache.spark.sql.DataFrame = [deptId: int, deptName: string ... 1 more field]
empLocation: String = /hyperspace/data-269515/employees.parquet
deptLocation: String = /hyperspace/data-269515/departments.parquet


Let's verify the contents of parquet files we created above to make sure they contain expected records in correct format. We later use these data files to create Hyperspace indexes and run sample queries.

Running below cell, the output displays the rows in employee and department dataframes in a tabular form. There should be 14 employees and 4 departments, each matching with one of triplets we created in the previous cell.

In [5]:
// empLocation and deptLocation are the user defined locations above to save parquet files
val empDF: DataFrame = spark.read.parquet(empLocation)
val deptDF: DataFrame = spark.read.parquet(deptLocation)

// Verify the data is available and correct
empDF.show()
deptDF.show()

StatementMeta(small, 17, 7, Finished, Available)

empDF: org.apache.spark.sql.DataFrame = [empId: int, empName: string ... 1 more field]
deptDF: org.apache.spark.sql.DataFrame = [deptId: int, deptName: string ... 1 more field]
+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7499|  ALLEN|    30|
| 7521|   WARD|    30|
| 7369|  SMITH|    20|
| 7844| TURNER|    30|
| 7876|  ADAMS|    20|
| 7900|  JAMES|    30|
| 7934| MILLER|    10|
| 7839|   KING|    10|
| 7566|  JONES|    20|
| 7698|  BLAKE|    30|
| 7782|  CLARK|    10|
| 7788|  SCOTT|    20|
| 7902|   FORD|    20|
| 7654| MARTIN|    30|
+-----+-------+------+

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+



## Hello Hyperspace Index!
Hyperspace lets users create indexes on records scanned from persisted data files. Once successfully created, an entry corresponding to the index is added to the Hyperspace's metadata. This metadata is later used by Apache Spark™'s Hyperspace-enabled optimizer during query processing to find and use proper indexes. 

Once indexes are created, users can perform several actions:
  - **Refresh** If the underlying data changes, users can refresh an existing index to capture that. 
  - **Delete** If the index is not needed, users can perform a soft-delete i.e., index is not physically deleted but is marked as 'deleted' so it is no longer used in your workloads.
  - **Vacuum** If an index is no longer required, users can vacuum it which forces a physical deletion of the index contents and associated metadata completely from Hyperspace's metadata.

Below sections show how such index management operations can be done in Hyperspace.

First, we need to import the required libraries and create an instance of Hyperspace. We later use this instance to invoke different Hyperspace APIs to create indexes on our sample data and modify those indexes.

Output of running below cell shows a reference to the created instance of Hyperspace.

In [6]:
// Create an instance of Hyperspace
import com.microsoft.hyperspace._

val hyperspace: Hyperspace = Hyperspace()

StatementMeta(small, 17, 8, Finished, Available)

import com.microsoft.hyperspace._
hyperspace: com.microsoft.hyperspace.Hyperspace = com.microsoft.hyperspace.Hyperspace@1b7f99d3


### Create Indexes

To create a Hyperspace index, the user needs to provide 2 pieces of information:
* An Apache Spark™ DataFrame which references the data to be indexed.
* An index configuration object: IndexConfig, which specifies the *index name*, *indexed* and *included* columns of the index. 

As you might have noticed, in this notebook, we illustrate indexing using the [Covering Index](https://www.red-gate.com/simple-talk/sql/learn-sql-server/using-covering-indexes-to-improve-query-performance/), which are the default index in Hyperspace. In the future, we plan on adding support for other index types. 

We start by creating three Hyperspace indexes on our sample data: two indexes on the department dataset named "deptIndex1" and "deptIndex2", and one index on the employee dataset named 'empIndex'. 
For each index, we need a corresponding IndexConfig to capture the name along with columns lists for the indexed and included columns. Running below cell creates these indexConfigs and its output lists them.

**Note**: An *index column* is a column that appears in your filters or join conditions. An *included column* is a column that appears in your select/project.

For instance, in the following query:
```sql
SELECT X
FROM Table
WHERE Y = 2
```
X can be an *index column* and Y can be an *included column*.

In [7]:
// Create index configurations
import com.microsoft.hyperspace.index.IndexConfig

val empIndexConfig: IndexConfig = IndexConfig("empIndex", Seq("deptId"), Seq("empName"))
val deptIndexConfig1: IndexConfig = IndexConfig("deptIndex1", Seq("deptId"), Seq("deptName"))
val deptIndexConfig2: IndexConfig = IndexConfig("deptIndex2", Seq("location"), Seq("deptName"))

StatementMeta(small, 17, 9, Finished, Available)

import com.microsoft.hyperspace.index.IndexConfig
empIndexConfig: com.microsoft.hyperspace.index.IndexConfig = [indexName: empIndex; indexedColumns: deptid; includedColumns: empname]
deptIndexConfig1: com.microsoft.hyperspace.index.IndexConfig = [indexName: deptIndex1; indexedColumns: deptid; includedColumns: deptname]
deptIndexConfig2: com.microsoft.hyperspace.index.IndexConfig = [indexName: deptIndex2; indexedColumns: location; includedColumns: deptname]


Now, we create three indexes using our index configurations. For this purpose, we invoke "createIndex" command on our Hyperspace instance. This command requires an index configuration and the dataFrame containing rows to be indexed.
Running below cell creates three indexes.


In [8]:
// Create indexes from configurations
import com.microsoft.hyperspace.index._

hyperspace.createIndex(empDF, empIndexConfig)
hyperspace.createIndex(deptDF, deptIndexConfig1)
hyperspace.createIndex(deptDF, deptIndexConfig2)

StatementMeta(small, 17, 10, Finished, Available)

import com.microsoft.hyperspace.index._


### List Indexes

Below code shows how a user can list all available indexes in a Hyperspace instance. It uses the `indexes` API which returns information about existing indexes as a Spark™'s DataFrame so you can perform additional operations. For instance, you can invoke valid operations on this DataFrame for checking its content or analyzing it further (for example filtering specific indexes or grouping them according to some desired property). 

Below cell uses DataFrame's `show` action to fully print the rows and show details of our indexes in a tabular form. For each index, we can see all the information Hyperspace has stored about it in its metadata. 

You will immediately notice the following:
  - `config.indexName`, `config.indexedColumns`, `config.includedColumns` are the fields that a user normally provides during index creation.
  - `status.status` indicates if the index is being actively used by the Spark's optimizer.
  - `dfSignature` is automatically generated by Hyperspace and is unique for each index. Hyperspace uses this signature internally to maintain the index and exploit it at query time. 
  
In the output below, all three indexes should have "ACTIVE" as status and their name, indexed columns, and included columns should match with what we defined in index configurations above.

In [9]:
hyperspace.indexes.show

StatementMeta(small, 17, 11, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://zhaotest@...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+



### Delete Indexes
A user can drop an existing index by using the `deleteIndex` API and providing the index name. 

Index deletion is a **soft-delete** operation i.e., only the index's status in the Hyperspace metadata from is changed from "ACTIVE" to "DELETED". This will exclude the deleted index from any future query optimization and Hyperspace no longer picks that index for any query. However, index files for a deleted index still remain available (since it is a soft-delete), so if you accidentally deleted the index, you could still restore it.

The cell below deletes index with name "deptIndex2" and lists Hyperspace metadata after that. The output should be similar to above cell for "List Indexes" except for "deptIndex2" which now should have its status changed into "DELETED".

In [10]:
hyperspace.deleteIndex("deptIndex2")

hyperspace.indexes.show

StatementMeta(small, 17, 12, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...| ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://zhaotest@...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+



### Restore Indexes
A user can use the `restoreIndex` API to restore a deleted index. This will bring back the latest version of index into ACTIVE status and makes it usable again for queries. 

The cell below shows an example of `restoreIndex` API. We delete "deptIndex1" and restore it. The output shows "deptIndex1" first went into the "DELETED" status after invoking "deleteIndex" command and came back to the "ACTIVE" status after calling "restoreIndex".

In [11]:
hyperspace.deleteIndex("deptIndex1")

hyperspace.indexes.show

hyperspace.restoreIndex("deptIndex1")

hyperspace.indexes.show

StatementMeta(small, 17, 13, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|DELETED|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://zhaotest@...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+

### Vacuum Indexes
The user can perform a **hard-delete** i.e., fully remove files and the metadata entry for a deleted index using the `vacuumIndex` API. Once done, this action is **irreversible** as it physically deletes all the index files associated with the index.

The cell below vacuums the "deptIndex2" index and shows Hyperspace metadata after vaccuming. You should see metadata entries for two indexes "deptIndex1" and "empIndex" both with "ACTIVE" status and no entry for "deptIndex2".

In [12]:
hyperspace.vacuumIndex("deptIndex2")

hyperspace.indexes.show

StatementMeta(small, 17, 14, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://zhaotest@...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://zhaotest@...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+



## Enable/Disable Hyperspace

Hyperspace provides APIs to enable or disable index usage with Spark™.

  - By using `enableHyperspace` API, Hyperspace optimization rules become visible to the Apache Spark™ optimizer and it will exploit existing Hyperspace indexes to optimize user queries.
  - By using `disableHyperspace` command, Hyperspace rules no longer apply during query optimization. You should note that disabling Hyperspace has no impact on created indexes as they remain intact.

Below cell shows how you can use these commands to enable or disable hyperspace. The output simply shows a reference to the existing Spark™ session whose configuration is updated.

In [13]:
// Enable Hyperspace
spark.enableHyperspace

// Disable Hyperspace
spark.disableHyperspace

StatementMeta(small, 17, 15, Finished, Available)

res59: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ab7b5f
res62: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ab7b5f


## Index Usage
In order to make Spark™ use Hyperspace indexes during query processing, the user needs to make sure that Hyperspace is enabled. 

The cell below enables Hyperspace and creates two DataFrames containing our sample data records which we use for running example queries. For each DataFrame, a few sample rows are printed.

In [14]:
// Enable Hyperspace
spark.enableHyperspace

val empDFrame: DataFrame = spark.read.parquet(empLocation)
val deptDFrame: DataFrame = spark.read.parquet(deptLocation)

empDFrame.show(5)
deptDFrame.show(5)

StatementMeta(small, 17, 16, Finished, Available)

res64: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ab7b5f
empDFrame: org.apache.spark.sql.DataFrame = [empId: int, empName: string ... 1 more field]
deptDFrame: org.apache.spark.sql.DataFrame = [deptId: int, deptName: string ... 1 more field]
+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7499|  ALLEN|    30|
| 7521|   WARD|    30|
| 7369|  SMITH|    20|
| 7844| TURNER|    30|
| 7876|  ADAMS|    20|
+-----+-------+------+
only showing top 5 rows

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+



# Hyperspace's Index Types

Currently, Hyperspace can exploit indexes for two groups of queries: 
* Selection queries with lookup or range selection filtering predicates.
* Join queries with an equality join predicate (i.e. Equi-joins).

## Indexes for Accelerating Filters

Our first example query does a lookup on department records (see below cell). In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId = 20
```

The output of running the cell below shows: 
- query result, which is a single department name.
- query plan that Spark™ used to run the query. 

In the query plan, the "FileScan" operator at the bottom of the plan shows the datasource where the records were read from. The location of this file indicates the path to the latest version of the "deptIndex1" index. This shows  that according to the query and using Hyperspace optimization rules, Spark™ decided to exploit the proper index at runtime.


In [15]:
// Filter with equality predicate

val eqFilter: DataFrame = deptDFrame.filter("deptId = 20").select("deptName")
eqFilter.show()

hyperspace.explain(eqFilter)(displayHTML(_))

StatementMeta(small, 17, 17, Finished, Available)

eqFilter: org.apache.spark.sql.DataFrame = [deptName: string]
+--------+
|deptName|
+--------+
|Research|
+--------+



============================================================= Plan with indexes: ============================================================= Project [deptName#527] +- Filter (isnotnull(deptId#526) && (deptId#526 = 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), EqualTo(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#527] +- Filter (isnotnull(deptId#526) && (deptId#526 = 20)) +- FileScan parquet [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), EqualTo(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=0

Our second example is a range selection query on department records. In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId > 20
```
Similar to our first example, the output of the cell below shows the query results (names of two departments) and the query plan. The location of data file in the FileScan operator shows that 'deptIndex1" was used to run the query.   


In [16]:
// Filter with range selection predicate

val rangeFilter: DataFrame = deptDFrame.filter("deptId > 20").select("deptName")
rangeFilter.show()

hyperspace.explain(rangeFilter)(displayHTML(_))

StatementMeta(small, 17, 18, Finished, Available)

rangeFilter: org.apache.spark.sql.DataFrame = [deptName: string]
+----------+
|  deptName|
+----------+
|Operations|
|     Sales|
+----------+



============================================================= Plan with indexes: ============================================================= Project [deptName#527] +- Filter (isnotnull(deptId#526) && (deptId#526 > 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#527] +- Filter (isnotnull(deptId#526) && (deptId#526 > 20)) +- FileScan parquet [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=0

Our third example is a query joining department and employee records on the department id. The equivalent SQL statement is shown below:

```sql
SELECT employees.deptId, empName, departments.deptId, deptName
FROM   employees, departments 
WHERE  employees.deptId = departments.deptId
```

The output of running the cell below shows the query results which are the names of 14 employees and the name of department each employee works in. The query plan is also included in the output. Notice how the file locations for two FileScan operators shows that Spark™ used "empIndex" and "deptIndex1" indexes to run the query.   


In [17]:
// Join

val eqJoin: DataFrame =
      empDFrame.
      join(deptDFrame, empDFrame("deptId") === deptDFrame("deptId")).
      select(empDFrame("empName"), deptDFrame("deptName"))

eqJoin.show()

hyperspace.explain(eqJoin)(displayHTML(_))

StatementMeta(small, 17, 19, Finished, Available)

eqJoin: org.apache.spark.sql.DataFrame = [empName: string, deptName: string]
+-------+----------+
|empName|  deptName|
+-------+----------+
|  SMITH|  Research|
|  JONES|  Research|
|   FORD|  Research|
|  ADAMS|  Research|
|  SCOTT|  Research|
|   KING|Accounting|
|  CLARK|Accounting|
| MILLER|Accounting|
|  JAMES|     Sales|
|  BLAKE|     Sales|
| MARTIN|     Sales|
|  ALLEN|     Sales|
|   WARD|     Sales|
| TURNER|     Sales|
+-------+----------+



============================================================= Plan with indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#522,empName#521] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/em..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#526, deptName#527] +- *(2) Filter isnotnull(deptId#526) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(2) Sort [deptId#522 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#522, 200), [id=#175] : +- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan parquet [empName#521,deptId#522] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/emplo..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#526 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#526, 200), [id=#181] +- *(3) Project [deptId#526, deptName#527] +- *(3) Filter isnotnull(deptId#526) +- *(3) FileScan parquet [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=0 empIndex:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/empIndex/v__=0

## Support for SQL Semantics

The index usage is transparent to whether the user uses DataFrame API or Spark™ SQL. The following example shows the same join example as before but using Spark SQL, showing the use of indexes if applicable.

In [18]:
empDFrame.createOrReplaceTempView("EMP")
deptDFrame.createOrReplaceTempView("DEPT")

val joinQuery = spark.sql("SELECT EMP.empName, DEPT.deptName FROM EMP, DEPT WHERE EMP.deptId = DEPT.deptId")

joinQuery.show()
hyperspace.explain(joinQuery)(displayHTML(_))

StatementMeta(small, 17, 20, Finished, Available)

joinQuery: org.apache.spark.sql.DataFrame = [empName: string, deptName: string]
+-------+----------+
|empName|  deptName|
+-------+----------+
|  SMITH|  Research|
|  JONES|  Research|
|   FORD|  Research|
|  ADAMS|  Research|
|  SCOTT|  Research|
|   KING|Accounting|
|  CLARK|Accounting|
| MILLER|Accounting|
|  JAMES|     Sales|
|  BLAKE|     Sales|
| MARTIN|     Sales|
|  ALLEN|     Sales|
|   WARD|     Sales|
| TURNER|     Sales|
+-------+----------+



============================================================= Plan with indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#522,empName#521] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/em..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#526, deptName#527] +- *(2) Filter isnotnull(deptId#526) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(2) Sort [deptId#522 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#522, 200), [id=#267] : +- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan parquet [empName#521,deptId#522] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/emplo..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#526 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#526, 200), [id=#273] +- *(3) Project [deptId#526, deptName#527] +- *(3) Filter isnotnull(deptId#526) +- *(3) FileScan parquet [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=0 empIndex:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/empIndex/v__=0

## Explain API

So far, you might have observed we have been using the explain API provided by Hyperspace. The `explain` API from Hyperspace is very similar to Spark's `df.explain` API but allows users to compare their original plan vs the updated index-dependent plan before running their query. You have an option to choose from html/plaintext/console mode to display the command output. 

The following cell shows an example with HTML. The highlighted section represents the difference between original and updated plans along with the indexes being used.

In [19]:
spark.conf.set("spark.hyperspace.explain.displayMode", "html")
hyperspace.explain(eqJoin)(displayHTML(_))

StatementMeta(small, 17, 21, Finished, Available)

============================================================= Plan with indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#522,empName#521] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/em..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#526, deptName#527] +- *(2) Filter isnotnull(deptId#526) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#521, deptName#527] +- SortMergeJoin [deptId#522], [deptId#526], Inner :- *(2) Sort [deptId#522 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#522, 200), [id=#329] : +- *(1) Project [empName#521, deptId#522] : +- *(1) Filter isnotnull(deptId#522) : +- *(1) FileScan parquet [empName#521,deptId#522] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/emplo..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#526 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#526, 200), [id=#335] +- *(3) Project [deptId#526, deptName#527] +- *(3) Filter isnotnull(deptId#526) +- *(3) FileScan parquet [deptId#526,deptName#527] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=0 empIndex:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/empIndex/v__=0

## Refresh Indexes
If the original data on which an index was created changes, then the index will no longer capture the latest state of data and hence will not be used by Hyperspace to provide any acceleration. The user can refresh such a stale index using the `refreshIndex` API. This causes the index to be fully rebuilt and updates it according to the latest data records.
    
    Spoiler alert: if you are worried about fully rebuilding your index every time your data changes, don't worry! We will show you how to *incrementally refresh* your index in subsequent cells below.

The two cells below show an example for this scenario:
- First cell adds two more departments to the original departments data. It reads and prints list of departments to verify new departments are added correctly. The output shows 6 departments in total: four old ones and two new. Invoking "refreshIndex" updates "deptIndex1" so index captures new departments.
- Second cell runs our range selection query example. The results should now contain four departments: two are the ones, seen before when we ran the query above, and two are the new departments we just added.

In [20]:
val extraDepartments = Seq(
      (50, "Inovation", "Seattle"),
	  (60, "Human Resources", "San Francisco"))
	  
val extraDeptData: DataFrame = extraDepartments.toDF("deptId", "deptName", "location")
extraDeptData.write.mode("Append").parquet(deptLocation)

val deptDFrameUpdated: DataFrame = spark.read.parquet(deptLocation)

deptDFrameUpdated.show(10)

hyperspace.refreshIndex("deptIndex1")

StatementMeta(small, 17, 22, Finished, Available)

extraDepartments: Seq[(Int, String, String)] = List((50,Inovation,Seattle), (60,Human Resources,San Francisco))
extraDeptData: org.apache.spark.sql.DataFrame = [deptId: int, deptName: string ... 1 more field]
deptDFrameUpdated: org.apache.spark.sql.DataFrame = [deptId: int, deptName: string ... 1 more field]
+------+---------------+-------------+
|deptId|       deptName|     location|
+------+---------------+-------------+
|    60|Human Resources|San Francisco|
|    10|     Accounting|     New York|
|    50|      Inovation|      Seattle|
|    40|     Operations|       Boston|
|    20|       Research|       Dallas|
|    30|          Sales|      Chicago|
+------+---------------+-------------+



In [21]:
val newRangeFilter: DataFrame = deptDFrameUpdated.filter("deptId > 20").select("deptName")
newRangeFilter.show()

hyperspace.explain(newRangeFilter)(displayHTML(_))

StatementMeta(small, 17, 23, Finished, Available)

newRangeFilter: org.apache.spark.sql.DataFrame = [deptName: string]
+---------------+
|       deptName|
+---------------+
|Human Resources|
|      Inovation|
|     Operations|
|          Sales|
+---------------+



============================================================= Plan with indexes: ============================================================= Project [deptName#938] +- Filter (isnotnull(deptId#937) && (deptId#937 > 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 7) [deptId#937,deptName#938] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/de..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#938] +- Filter (isnotnull(deptId#937) && (deptId#937 > 20)) +- FileScan parquet [deptId#937,deptName#938] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/depar..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/deptIndex1/v__=1

## Hybrid Scan for Mutable Datasets

Often times, if your underlying source data had some new files appended or existing files deleted, your index will get stale and Hyperspace decides not to use it. However, there are times where you just want to use the index without having to refresh it everytime. There could be multiple reasons for doing so:

  1. You do not want to continuosly refresh your index but instead want to do it periodically since you understand your workloads the best.  
  2. You added/removed only a few files and do not want to wait for yet another refresh job to finish. 

To allow you to still use a stale index, Hyperspace introduces **Hybrid Scan**, a novel technique that allows users to utilize outdated or stale indexes (e.g., the underlying source data had some new files appended or existing files deleted), without refreshing indexes. 

To achieve this, when you set the appropriate configuration to enable Hybrid Scan, Hyperspace modifies the query plan to leverage the changes as following:
- Appended files can be merged to index data by using **`Union` or `BucketUnion` (for join)**. Shuffling appended data can also be applied before merging, if needed.
- Deleted files can be handled by injecting `Filter-NOT-IN` condition on **lineage column** of index data, so that the indexed rows from the deleted files can be excluded at query time. 

You can check the transformation of the query plan in below examples.

    Note: Hybrid scan is only supported for non-partitioned data. Support for partitioned data is currently being worked upon.

### Hybrid Scan for appended files - non-partitioned data

Non-partitioned data is used in below example. In this example, we expect Join index can be used for the query and `BucketUnion` is introduced for appended files.

In [22]:
// GENERATE TEST DATA

val testData = Seq(
    ("orange", 3, "2020-10-01"),
    ("banana", 1, "2020-10-01"),
    ("carrot", 5, "2020-10-02"),
    ("beetroot", 12, "2020-10-02"),
    ("orange", 2, "2020-10-03"),
    ("banana", 11, "2020-10-03"),
    ("carrot", 3, "2020-10-03"),
    ("beetroot", 2, "2020-10-04"),
    ("cucumber", 7, "2020-10-05"),
    ("pepper", 20, "2020-10-06")
    ).toDF("name", "qty", "date")

val testDataLocation = s"$dataPath/productTable"

testData.write.mode("overwrite").parquet(testDataLocation)
val testDF = spark.read.parquet(testDataLocation)

StatementMeta(small, 17, 24, Finished, Available)

testData: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 1 more field]
testDataLocation: String = /hyperspace/data-269515/productTable
testDF: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 1 more field]


In [23]:
// CREATE INDEX
hyperspace.createIndex(testDF, IndexConfig("productIndex2", Seq("name"), Seq("date", "qty")))

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
val filter1 = testDF.filter("name = 'banana'")
val filter2 = testDF.filter("qty > 10")
val query = filter1.join(filter2, "name")

// Check Join index rule is applied properly.
hyperspace.explain(query)(displayHTML(_))

StatementMeta(small, 17, 25, Finished, Available)

filter1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, qty: int ... 1 more field]
filter2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, qty: int ... 1 more field]
query: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 3 more fields]


============================================================= Plan with indexes: ============================================================= Project [name#1056, qty#1057, date#1058, qty#1081, date#1082] +- SortMergeJoin [name#1056], [name#1080], Inner :- *(1) Project [name#1056, qty#1057, date#1058] : +- *(1) Filter (isnotnull(name#1056) && (name#1056 = banana)) : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 1) [name#1056,date#1058,qty#1057] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- *(2) Project [name#1080, qty#1081, date#1082] +- *(2) Filter (((isnotnull(qty#1081) && (qty#1081 > 10)) && (name#1080 = banana)) && isnotnull(name#1080)) +- *(2) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 1) [name#1080,date#1082,qty#1081] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 ============================================================= Plan without indexes: ============================================================= Project [name#1056, qty#1057, date#1058, qty#1081, date#1082] +- SortMergeJoin [name#1056], [name#1080], Inner :- *(2) Sort [name#1056 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1056, 200), [id=#453] : +- *(1) Project [name#1056, qty#1057, date#1058] : +- *(1) Filter (isnotnull(name#1056) && (name#1056 = banana)) : +- *(1) FileScan parquet [name#1056,qty#1057,date#1058] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1080 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1080, 200), [id=#459] +- *(3) Project [name#1080, qty#1081, date#1082] +- *(3) Filter (((isnotnull(qty#1081) && (qty#1081 > 10)) && (name#1080 = banana)) && isnotnull(name#1080)) +- *(3) FileScan parquet [name#1080,qty#1081,date#1082] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Indexes used: ============================================================= productIndex2:abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/productIndex2/v__=0

In [24]:
// Append new files.
val appendData = Seq(
    ("orange", 13, "2020-11-01"),
    ("banana", 5, "2020-11-01")).toDF("name", "qty", "date")
appendData.write.mode("append").parquet(testDataLocation)

StatementMeta(small, 17, 26, Finished, Available)

appendData: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 1 more field]


Hybrid scan is disabled by default. Therefore, you will see that since we appended new data, Hyperspace will decide NOT to use the index.

In the output, you will see no plan differences (hence no highlighting).

In [25]:
// Hybrid Scan configs are false by default.
spark.conf.set("spark.hyperspace.index.hybridscan.enabled", "false")
spark.conf.set("spark.hyperspace.index.hybridscan.delete.enabled", "false")

val testDFWithAppend = spark.read.parquet(testDataLocation)
val filter1 = testDFWithAppend.filter("name = 'banana'")
val filter2 = testDFWithAppend.filter("qty > 10")
val query = filter1.join(filter2, "name")
hyperspace.explain(query)(displayHTML(_))
query.show

StatementMeta(small, 17, 27, Finished, Available)

testDFWithAppend: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 1 more field]
filter1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, qty: int ... 1 more field]
filter2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, qty: int ... 1 more field]
query: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 3 more fields]


============================================================= Plan with indexes: ============================================================= Project [name#1172, qty#1173, date#1174, qty#1179, date#1180] +- SortMergeJoin [name#1172], [name#1178], Inner :- *(2) Sort [name#1172 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1172, 200), [id=#589] : +- *(1) Project [name#1172, qty#1173, date#1174] : +- *(1) Filter (isnotnull(name#1172) && (name#1172 = banana)) : +- *(1) FileScan parquet [name#1172,qty#1173,date#1174] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1178 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1178, 200), [id=#595] +- *(3) Project [name#1178, qty#1179, date#1180] +- *(3) Filter (((isnotnull(qty#1179) && (qty#1179 > 10)) && (name#1178 = banana)) && isnotnull(name#1178)) +- *(3) FileScan parquet [name#1178,qty#1179,date#1180] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [name#1172, qty#1173, date#1174, qty#1179, date#1180] +- SortMergeJoin [name#1172], [name#1178], Inner :- *(2) Sort [name#1172 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1172, 200), [id=#536] : +- *(1) Project [name#1172, qty#1173, date#1174] : +- *(1) Filter (isnotnull(name#1172) && (name#1172 = banana)) : +- *(1) FileScan parquet [name#1172,qty#1173,date#1174] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1178 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1178, 200), [id=#542] +- *(3) Project [name#1178, qty#1179, date#1180] +- *(3) Filter (((isnotnull(qty#1179) && (qty#1179 > 10)) && (name#1178 = banana)) && isnotnull(name#1178)) +- *(3) FileScan parquet [name#1178,qty#1179,date#1180] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Indexes used: =============================================================


+------+---+----------+---+----------+
|  name|qty|      date|qty|      date|
+------+---+----------+---+----------+
|banana|  5|2020-11-01| 11|2020-10-03|
|banana|  1|2020-10-01| 11|2020-10-03|
|banana| 11|2020-10-03| 11|2020-10-03|
+------+---+----------+---+----------+



### Enable Hybrid Scan

In plan with indexes, you can see
`Exchange hashpartitioning` required only for appended files so that we could still utilize the "shuffled" index data with appended files. `BucketUnion` is used to merge "shuffled" appended files with the index data.

In [26]:
// Enable Hybrid Scan config. "delete" config is not necessary since we only appended data.
spark.conf.set("spark.hyperspace.index.hybridscan.enabled", "true")
// spark.conf.set("spark.hyperspace.index.hybridscan.delete.enabled", "true")
spark.enableHyperspace
// Need to redefine query to recalculate the query plan.
val query = filter1.join(filter2, "name")
hyperspace.explain(query)(displayHTML(_))
query.show

StatementMeta(small, 17, 28, Finished, Available)

res126: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ab7b5f
query: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 3 more fields]


============================================================= Plan with indexes: ============================================================= Project [name#1172, qty#1173, date#1174, qty#1271, date#1272] +- SortMergeJoin [name#1172], [name#1270], Inner :- *(3) Sort [name#1172 ASC NULLS FIRST], false, 0 : +- BucketUnion 200 buckets, bucket columns: [name] : :- *(1) Project [name#1172, qty#1173, date#1174] : : +- *(1) Filter (isnotnull(name#1172) && (name#1172 = banana)) : : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 1) [name#1172,date#1174,qty#1173] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 : +- Exchange hashpartitioning(name#1172, 200), [id=#775] : +- *(2) Project [name#1172, qty#1173, date#1174] : +- *(2) Filter (isnotnull(name#1172) && (name#1172 = banana)) : +- *(2) FileScan parquet [name#1172,qty#1173,date#1174] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(6) Sort [name#1270 ASC NULLS FIRST], false, 0 +- BucketUnion 200 buckets, bucket columns: [name] :- *(4) Project [name#1270, qty#1271, date#1272] : +- *(4) Filter (((isnotnull(qty#1271) && (qty#1271 > 10)) && isnotnull(name#1270)) && (name#1270 = banana)) : +- *(4) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 1) [name#1270,date#1272,qty#1271] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- Exchange hashpartitioning(name#1270, 200), [id=#783] +- *(5) Project [name#1270, qty#1271, date#1272] +- *(5) Filter (((isnotnull(qty#1271) && (qty#1271 > 10)) && isnotnull(name#1270)) && (name#1270 = banana)) +- *(5) FileScan parquet [name#1270,qty#1271,date#1272] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [name#1172, qty#1173, date#1174, qty#1271, date#1272] +- SortMergeJoin [name#1172], [name#1270], Inner :- *(2) Sort [name#1172 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1172, 200), [id=#701] : +- *(1) Project [name#1172, qty#1173, date#1174] : +- *(1) Filter (isnotnull(name#1172) && (name#1172 = banana)) : +- *(1) FileScan parquet [name#1172,qty#1173,date#1174] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1270 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1270, 200), [id=#707] +- *(3) Project [name#1270, qty#1271, date#1272] +- *(3) Filter (((isnotnull(qty#1271) && (qty#1271 > 10)) && isnotnull(name#1270)) && (name#1270 = banana)) +- *(3) FileScan parquet [name#1270,qty#1271,date#1272] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ==========================================================


+------+---+----------+---+----------+
|  name|qty|      date|qty|      date|
+------+---+----------+---+----------+
|banana|  1|2020-10-01| 11|2020-10-03|
|banana| 11|2020-10-03| 11|2020-10-03|
|banana|  5|2020-11-01| 11|2020-10-03|
+------+---+----------+---+----------+



## Incremental Index Refresh
When you ready to update your indexes but do not want to rebuild your entire index, Hyperspace supports updating indexes in an incremental manner using `hs.refreshIndex("name", "incremental")` API. This will allow eliminate the need for a full rebuild of index from scratch, utilizing previously created index files as well as updating indexes on only the newly added data.

Of course, please be sure to use the complementary `optimizeIndex` API (shown below) periodically to make sure you do not see performance regressions. We recommend calling `optimize` at least once for every 10 times you call `refreshIndex(..., "incremental")`, assuming the data you added/removed is < 10% of the original dataset. For instance, if your original dataset is 100 GB, and you've added/removed data in increments/decrements of 1 GB, you can call `refreshIndex` 10 times before calling `optimizeIndex`. Please note that this example is simply used for illustration and you have to adapt this for your workloads.

In the example below, notice the addition of a `Sort` node in the query plan when indexes are used. This is because partial indexes are created on the appended data files, causing Spark to introduce a `Sort`. Please also note that `Shuffle` i.e. `Exchange` is still eliminated from the plan, giving you the appropriate acceleration.

In [27]:
def query(): DataFrame = {
    val testDFWithAppend = spark.read.parquet(testDataLocation)
    val filter1 = testDFWithAppend.filter("name = 'banana'")
    val filter2 = testDFWithAppend.filter("qty > 10")
    filter1.join(filter2, "name")
}

hyperspace.refreshIndex("productIndex2", "incremental")

hyperspace.explain(query())(displayHTML(_))
query().show

StatementMeta(small, 17, 29, Finished, Available)

query: ()org.apache.spark.sql.DataFrame


============================================================= Plan with indexes: ============================================================= Project [name#1392, qty#1393, date#1394, qty#1399, date#1400] +- SortMergeJoin [name#1392], [name#1398], Inner :- *(1) Sort [name#1392 ASC NULLS FIRST], false, 0 : +- *(1) Project [name#1392, qty#1393, date#1394] : +- *(1) Filter (isnotnull(name#1392) && (name#1392 = banana)) : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 3) [name#1392,date#1394,qty#1393] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- *(2) Sort [name#1398 ASC NULLS FIRST], false, 0 +- *(2) Project [name#1398, qty#1399, date#1400] +- *(2) Filter (((isnotnull(qty#1399) && (qty#1399 > 10)) && (name#1398 = banana)) && isnotnull(name#1398)) +- *(2) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 3) [name#1398,date#1400,qty#1399] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 ============================================================= Plan without indexes: ============================================================= Project [name#1392, qty#1393, date#1394, qty#1399, date#1400] +- SortMergeJoin [name#1392], [name#1398], Inner :- *(2) Sort [name#1392 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1392, 200), [id=#927] : +- *(1) Project [name#1392, qty#1393, date#1394] : +- *(1) Filter (isnotnull(name#1392) && (name#1392 = banana)) : +- *(1) FileScan parquet [name#1392,qty#1393,date#1394] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1398 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1398, 200), [id=#933] +- *(3) Project [name#1398, qty#1399, date#1400] +- *(3) Filter (((isnotnull(qty#1399) && (qty#1399 > 10)) && (name#1398 = banana)) && isnotnull(name#1398)) +- *(3) FileScan parquet [name#1398,qty#1399,date#1400] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Indexes used: =============================================================


+------+---+----------+---+----------+
|  name|qty|      date|qty|      date|
+------+---+----------+---+----------+
|banana|  1|2020-10-01| 11|2020-10-03|
|banana| 11|2020-10-03| 11|2020-10-03|
|banana|  5|2020-11-01| 11|2020-10-03|
+------+---+----------+---+----------+



## Optimize Index layout
After calling incremental refreshes multiple times on newly appended data (e.g. if the user writes to data in small batches or in case of streaming scenarios), the number of index files tend to become large affecting the performance of the index (large number of small files problem). Hyperspace provides `hyperspace.optimizeIndex("indexName")` API to optimize the index layout and reduce the large files problem.

In the plan below, notice that Hyperspace has removed the additional `Sort` node in the query plan. Optimize can help avoiding sorting for any index bucket which contains only one file. However, this will only be true if ALL the index buckets have at most 1 file per bucket, after `optimizeIndex`.

In [28]:
// Append some more data and call refresh again.
val appendData = Seq(
    ("orange", 13, "2020-11-01"),
    ("banana", 5, "2020-11-01")).toDF("name", "qty", "date")
appendData.write.mode("append").parquet(testDataLocation)

hyperspace.refreshIndex("productIndex2", "incremental")

StatementMeta(small, 17, 30, Finished, Available)

appendData: org.apache.spark.sql.DataFrame = [name: string, qty: int ... 1 more field]


In [29]:
// Call optimize. Ensure that Sort is removed after optimization (This is possible here because after optimize, in this case, every bucket contains only 1 file.).
hyperspace.optimizeIndex("productIndex2")

hyperspace.explain(query())(displayHTML(_))

StatementMeta(small, 17, 31, Finished, Available)

============================================================= Plan with indexes: ============================================================= Project [name#1559, qty#1560, date#1561, qty#1566, date#1567] +- SortMergeJoin [name#1559], [name#1565], Inner :- *(1) Project [name#1559, qty#1560, date#1561] : +- *(1) Filter (isnotnull(name#1559) && (name#1559 = banana)) : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 7) [name#1559,date#1561,qty#1560] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- *(2) Project [name#1565, qty#1566, date#1567] +- *(2) Filter (((isnotnull(qty#1566) && (qty#1566 > 10)) && isnotnull(name#1565)) && (name#1565 = banana)) +- *(2) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 7) [name#1565,date#1567,qty#1566] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 ============================================================= Plan without indexes: ============================================================= Project [name#1559, qty#1560, date#1561, qty#1566, date#1567] +- SortMergeJoin [name#1559], [name#1565], Inner :- *(2) Sort [name#1559 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1559, 200), [id=#1070] : +- *(1) Project [name#1559, qty#1560, date#1561] : +- *(1) Filter (isnotnull(name#1559) && (name#1559 = banana)) : +- *(1) FileScan parquet [name#1559,qty#1560,date#1561] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1565 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1565, 200), [id=#1076] +- *(3) Project [name#1565, qty#1566, date#1567] +- *(3) Filter (((isnotnull(qty#1566) && (qty#1566 > 10)) && isnotnull(name#1565)) && (name#1565 = banana)) +- *(3) FileScan parquet [name#1565,qty#1566,date#1567] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Indexes used: =============================================================

#### Optimize modes

The default mode for optimization is "quick" mode where files smaller than a predefined threshold are picked for optmization. To maximize the effect of optimization, Hyperspace allows another optimize mode "full" as shown below. This mode picks ALL index files for optimization irrespective of their file size and creates the best possible layout of the index. This is also slower than the default optimize mode because more data is being processed here.

In [30]:
hyperspace.optimizeIndex("productIndex2", "full")

hyperspace.explain(query())(displayHTML(_))

StatementMeta(small, 17, 32, Finished, Available)

============================================================= Plan with indexes: ============================================================= Project [name#1641, qty#1642, date#1643, qty#1648, date#1649] +- SortMergeJoin [name#1641], [name#1647], Inner :- *(1) Project [name#1641, qty#1642, date#1643] : +- *(1) Filter (isnotnull(name#1641) && (name#1641 = banana)) : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 7) [name#1641,date#1643,qty#1642] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- *(2) Project [name#1647, qty#1648, date#1649] +- *(2) Filter (((isnotnull(qty#1648) && (qty#1648 > 10)) && isnotnull(name#1647)) && (name#1647 = banana)) +- *(2) FileScan Hyperspace(Type: CI, Name: productIndex2, LogVersion: 7) [name#1647,date#1649,qty#1648] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/indexes-269515/pr..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 ============================================================= Plan without indexes: ============================================================= Project [name#1641, qty#1642, date#1643, qty#1648, date#1649] +- SortMergeJoin [name#1641], [name#1647], Inner :- *(2) Sort [name#1641 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1641, 200), [id=#1148] : +- *(1) Project [name#1641, qty#1642, date#1643] : +- *(1) Filter (isnotnull(name#1641) && (name#1641 = banana)) : +- *(1) FileScan parquet [name#1641,qty#1642,date#1643] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1647 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1647, 200), [id=#1154] +- *(3) Project [name#1647, qty#1648, date#1649] +- *(3) Filter (((isnotnull(qty#1648) && (qty#1648 > 10)) && isnotnull(name#1647)) && (name#1647 = banana)) +- *(3) FileScan parquet [name#1647,qty#1648,date#1649] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://zhaotest@bdbjtestgen2.dfs.core.windows.net/hyperspace/data-269515/produ..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Indexes used: =============================================================

### Clean Up
To make this notebook self-contained and not leave any dangling data, we have some small clean-up code below. 

In [31]:
mssparkutils.fs.rm(dataPath, true)
mssparkutils.fs.rm(indexLocation, true)

StatementMeta(small, 17, 33, Finished, Available)

res146: Boolean = true
res147: Boolean = true
